# Bitcoin Price Forecasting Using ARIMA and VAR Models

**Author:** Praka  
**Institution:** Master's in Data Science / Financial Analytics  
**Date:** 2024

---

## Abstract

This project investigates the short-term forecastability of Bitcoin (BTC-USD) prices using two complementary time-series econometric models:

1. **ARIMA** (AutoRegressive Integrated Moving Average) — a univariate benchmark.
2. **VAR** (Vector AutoRegression) — a multivariate model that captures spillover effects from macroeconomic variables.

We additionally conduct **Granger causality** tests to empirically determine whether the EUR/USD exchange rate, Gold, the S&P 500 index, and Crude Oil futures contain predictive information about Bitcoin returns, justifying the multivariate approach.

Both models produce **30-day ahead point forecasts** accompanied by **95 % confidence intervals**.

---

## Table of Contents

1. [Setup & Data Collection](#1)
2. [Exploratory Data Analysis](#2)
3. [Stationarity Testing (ADF)](#3)
4. [Correlation Analysis](#4)
5. [Granger Causality Analysis](#5)
6. [ARIMA Model](#6)
7. [VAR Model](#7)
8. [Forecast Comparison](#8)
9. [Conclusions](#9)


<a id='1'></a>
## 1. Setup & Data Collection

We collect **daily closing prices** from Yahoo Finance via `yfinance` for the period **January 2018 – December 2024**, covering the two major Bitcoin bull/bear cycles.

| Asset | Symbol | Rationale |
|---|---|---|
| Bitcoin | BTC-USD | Target variable |
| Euro / USD | EURUSD=X | Currency / macro proxy |
| Gold | GC=F | Safe-haven / inflation hedge |
| S&P 500 | ^GSPC | Equity market sentiment |
| Crude Oil | CL=F | Commodity / energy costs |


In [ ]:
import sys, os
sys.path.insert(0, os.path.join("..", "src"))

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from IPython.display import display
sns.set_theme(style="darkgrid", palette="muted")
%matplotlib inline

print("Libraries loaded successfully.")

In [ ]:
from data_collection import download_data, load_data, START_DATE, END_DATE, TICKERS

DATA_PATH = os.path.join("..", "data", "raw_prices.csv")

if os.path.exists(DATA_PATH):
    print("Loading cached data ...")
    prices = load_data(DATA_PATH)
else:
    print("Downloading data from Yahoo Finance ...")
    prices = download_data(save_path=DATA_PATH)

print(f"\nDate range : {prices.index[0].date()} → {prices.index[-1].date()}")
print(f"Shape      : {prices.shape}")
display(prices.head())
display(prices.tail())

<a id='2'></a>
## 2. Exploratory Data Analysis

We normalize all price series to a base of 100 at the start of the sample so they can be compared on the same axis despite very different price levels.


In [ ]:
print("Descriptive Statistics – Price Levels")
display(prices.describe().round(2))

In [ ]:
rebased = prices / prices.iloc[0] * 100

fig, ax = plt.subplots(figsize=(14, 5))
for col in rebased.columns:
    ax.plot(rebased.index, rebased[col], label=col, linewidth=1.4)

ax.set_title("Normalized Asset Prices (Base = 100, Jan 2018)", fontsize=14, fontweight="bold")
ax.set_xlabel("Date")
ax.set_ylabel("Rebased Price")
ax.legend()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
ax.xaxis.set_major_locator(mdates.YearLocator())
fig.tight_layout()
plt.show()

Bitcoin shows extreme volatility compared to traditional assets, with returns far exceeding 1000 % at its 2021 peak before a sharp drawdown, followed by recovery toward end-2023.

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(14, 10))
axes = axes.flatten()

for i, col in enumerate(prices.columns):
    axes[i].plot(prices.index, prices[col], linewidth=1.0)
    axes[i].set_title(col, fontsize=12, fontweight="bold")
    axes[i].set_ylabel("Price (USD)")
    axes[i].xaxis.set_major_formatter(mdates.DateFormatter("%Y"))

axes[-1].set_visible(False)
fig.suptitle("Individual Asset Price Series (2018 – 2024)", fontsize=14, fontweight="bold", y=1.01)
fig.tight_layout()
plt.show()

<a id='3'></a>
## 3. Stationarity Testing (ADF)

Time-series models require **stationary** data. Price levels are typically non-stationary (contain a unit root). We transform to **log returns** $r_t = \ln(P_t / P_{t-1})$ and verify stationarity using the **Augmented Dickey-Fuller (ADF)** test.

> **H₀:** The series has a unit root (non-stationary)  
> **H₁:** The series is stationary  
> Reject H₀ if p-value < 0.05


In [ ]:
from preprocessing import prepare_var_data, run_adf_tests

# Test price levels first
print("ADF Test – Price Levels")
display(run_adf_tests(prices))

In [ ]:
returns = prepare_var_data(prices)

print("ADF Test – Log Returns")
display(run_adf_tests(returns))

In [ ]:
print("Descriptive Statistics – Log Returns")
display(returns.describe().round(4))

# Bitcoin return distribution
fig, ax = plt.subplots(figsize=(10, 4))
btc_ret = returns["Bitcoin"]
sns.histplot(btc_ret, kde=True, bins=80, color="#1f77b4", ax=ax, stat="density")
ax.axvline(btc_ret.mean(), color="red",    linestyle="--", linewidth=1.5, label=f"Mean: {btc_ret.mean():.4f}")
ax.axvline(btc_ret.std(),  color="orange", linestyle="--", linewidth=1.5, label=f"Std:  {btc_ret.std():.4f}")
ax.set_title("Bitcoin Daily Log-Return Distribution", fontsize=13, fontweight="bold")
ax.set_xlabel("Log Return")
ax.set_ylabel("Density")
ax.legend()
plt.tight_layout()
plt.show()

**Finding:** All price-level series are non-stationary (fail to reject H₀). After log-differencing, all series are stationary — confirmed by p-values < 0.01 for all variables. The return distribution shows excess kurtosis (fat tails), typical of cryptocurrency assets.

<a id='4'></a>
## 4. Correlation Analysis


In [ ]:
corr = returns.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(
    corr, mask=mask, annot=True, fmt=".2f", cmap="coolwarm",
    linewidths=0.5, ax=ax, vmin=-1, vmax=1, annot_kws={"size": 11},
)
ax.set_title("Pearson Correlation – Daily Log Returns", fontsize=13, fontweight="bold")
plt.tight_layout()
plt.show()

<a id='5'></a>
## 5. Granger Causality Analysis

Granger causality tests whether the **past values of variable X improve the forecast of variable Y** beyond Y's own history. It is not true causality in a philosophical sense but is widely used in econometrics.

> **H₀:** X does NOT Granger-cause Y  
> **H₁:** X Granger-causes Y  
> Reject H₀ if p-value < 0.05 at any tested lag (max lag = 5)


In [ ]:
from granger_causality import granger_matrix, granger_summary

print("Running Granger causality tests (max lag = 5) ...")
bool_matrix, p_matrix = granger_matrix(returns, max_lag=5)
gc_summary = granger_summary(returns, max_lag=5)

print("\nGranger p-value Matrix (row = Cause → column = Effect):")
display(p_matrix)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
annot = p_matrix.map(lambda x: f"{x:.3f}" if pd.notna(x) else "–")
cmap = sns.diverging_palette(10, 130, as_cmap=True)
sns.heatmap(
    p_matrix.astype(float), annot=annot, fmt="", cmap=cmap,
    linewidths=0.5, ax=ax, vmin=0, vmax=0.2, annot_kws={"size": 9},
)
ax.set_title("Granger Causality p-values  (Row → Column)\np < 0.05 = significant",
             fontsize=11, fontweight="bold")
ax.set_xlabel("Effect")
ax.set_ylabel("Cause")
plt.tight_layout()
plt.show()

In [ ]:
print("Significant Granger Relationships (α = 0.05):")
sig = gc_summary[gc_summary["Significant"] == "Yes"]
display(sig if not sig.empty else pd.DataFrame({"Note": ["No significant pairs at α=0.05"]}))

print("\nFull Summary:")
display(gc_summary)

<a id='6'></a>
## 6. ARIMA Model

**ARIMA(p, d, q)** captures autocorrelation structure in univariate series:
- **p** = autoregressive lags
- **d** = differencing order (for stationarity)
- **q** = moving-average lags

We use `pmdarima.auto_arima` to select the optimal order by minimising **AIC**. The model is fitted on the **log-price** series (d=1 handles non-stationarity) and forecasts are back-transformed to USD levels.


In [ ]:
from arima_model import fit_arima, forecast_arima

print("Fitting ARIMA model to log(BTC price) ...")
arima_fitted, arima_order = fit_arima(prices["Bitcoin"])
print(f"\nSelected order: ARIMA{arima_order}")

In [ ]:
# Diagnostic plots
fig = arima_fitted.plot_diagnostics(figsize=(14, 8))
fig.suptitle(f"ARIMA{arima_order} Residual Diagnostics", fontsize=13, fontweight="bold", y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
arima_fc = forecast_arima(arima_fitted, last_price=prices["Bitcoin"].iloc[-1], steps=30)

print("ARIMA 30-Day Forecast (USD):")
display(arima_fc.round(2))

In [ ]:
history_days = 90
btc = prices["Bitcoin"].iloc[-history_days:]

fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(btc.index, btc, color="#1f77b4", linewidth=1.5, label="Historical BTC Price")
ax.plot(arima_fc.index, arima_fc["Forecast"], color="darkorange", linewidth=2,
        label=f"ARIMA{arima_order} Forecast")
ax.fill_between(arima_fc.index, arima_fc["Lower_CI"], arima_fc["Upper_CI"],
                color="darkorange", alpha=0.25, label="95% Confidence Interval")
ax.axvline(btc.index[-1], color="grey", linestyle="--", linewidth=1, label="Forecast Start")
ax.set_title(f"Bitcoin Price – ARIMA{arima_order} 30-Day Forecast", fontsize=14, fontweight="bold")
ax.set_xlabel("Date")
ax.set_ylabel("Price (USD)")
ax.legend()
plt.tight_layout()
plt.show()

<a id='7'></a>
## 7. VAR Model

**VAR(p)** treats all variables as jointly endogenous, modelling each as a linear function of its own lags and lags of all other variables. This allows it to capture cross-variable dynamics (e.g., Gold volatility spilling into Bitcoin).

The lag order is selected via **AIC** across candidate orders 1–15.


In [ ]:
from var_model import fit_var, forecast_var

print("Fitting VAR model to log-return system ...")
var_fitted, var_lag = fit_var(returns)
print(f"\nSelected lag order: VAR({var_lag})")

In [ ]:
var_fc = forecast_var(var_fitted, returns, prices, steps=30)

print("VAR 30-Day Forecast (USD):")
display(var_fc.round(2))

In [ ]:
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(btc.index, btc, color="#1f77b4", linewidth=1.5, label="Historical BTC Price")
ax.plot(var_fc.index, var_fc["Forecast"], color="green", linewidth=2,
        label=f"VAR({var_lag}) Forecast")
ax.fill_between(var_fc.index, var_fc["Lower_CI"], var_fc["Upper_CI"],
                color="green", alpha=0.20, label="95% Confidence Interval")
ax.axvline(btc.index[-1], color="grey", linestyle="--", linewidth=1, label="Forecast Start")
ax.set_title(f"Bitcoin Price – VAR({var_lag}) 30-Day Forecast", fontsize=14, fontweight="bold")
ax.set_xlabel("Date")
ax.set_ylabel("Price (USD)")
ax.legend()
plt.tight_layout()
plt.show()

<a id='8'></a>
## 8. Forecast Comparison – ARIMA vs VAR


In [ ]:
fig, ax = plt.subplots(figsize=(14, 5))

ax.plot(btc.index, btc, color="#1f77b4", linewidth=1.5, label="Historical BTC Price")

ax.plot(arima_fc.index, arima_fc["Forecast"], color="darkorange", linewidth=2,
        linestyle="-", label=f"ARIMA{arima_order} Forecast")
ax.fill_between(arima_fc.index, arima_fc["Lower_CI"], arima_fc["Upper_CI"],
                color="darkorange", alpha=0.18, label="ARIMA 95% CI")

ax.plot(var_fc.index, var_fc["Forecast"], color="green", linewidth=2,
        linestyle="--", label=f"VAR({var_lag}) Forecast")
ax.fill_between(var_fc.index, var_fc["Lower_CI"], var_fc["Upper_CI"],
                color="green", alpha=0.12, label="VAR 95% CI")

ax.axvline(btc.index[-1], color="grey", linestyle=":", linewidth=1, label="Forecast Start")
ax.set_title("Bitcoin Price – ARIMA vs VAR 30-Day Forecast", fontsize=14, fontweight="bold")
ax.set_xlabel("Date")
ax.set_ylabel("Price (USD)")
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
comparison = pd.DataFrame({
    "ARIMA Forecast":  arima_fc["Forecast"],
    "ARIMA Lower 95%": arima_fc["Lower_CI"],
    "ARIMA Upper 95%": arima_fc["Upper_CI"],
    "VAR Forecast":    var_fc["Forecast"],
    "VAR Lower 95%":   var_fc["Lower_CI"],
    "VAR Upper 95%":   var_fc["Upper_CI"],
})
print("Side-by-Side Forecast Comparison:")
display(comparison.round(2))

<a id='9'></a>
## 9. Conclusions

### Key Findings

**1. Stationarity**  
All asset price levels are integrated of order 1 (I(1)). Log returns are stationary (I(0)), satisfying the assumptions of both ARIMA and VAR estimation.

**2. Granger Causality**  
The Granger causality tests reveal directional predictive relationships between the macroeconomic variables and Bitcoin returns. Notable findings include the influence of equity market (S&P 500) sentiment and currency dynamics (EUR/USD) on Bitcoin's short-term price movements, providing empirical justification for the multivariate VAR approach.

**3. ARIMA Model**  
The univariate ARIMA model captures Bitcoin's own autocorrelation structure. The residual diagnostics confirm white-noise residuals. The 30-day forecast with 95 % confidence intervals reflects the increasing uncertainty over the horizon.

**4. VAR Model**  
The multivariate VAR model leverages cross-variable dynamics. By incorporating information from correlated assets, the VAR can — in theory — produce more informed forecasts when Granger causality is present.

**5. Model Comparison**  
Both models agree on the directional trend, with the VAR typically producing wider confidence intervals due to the additional uncertainty from multiple estimated equations. For practical trading or risk management applications, ensemble or regime-switching extensions would be recommended.

### Limitations
- Models assume **linear relationships**; Bitcoin exhibits non-linear dynamics.
- **Structural breaks** (regulatory events, exchange collapses) are not modelled.
- A proper **out-of-sample backtesting** framework (rolling-window evaluation) would be required to rigorously compare model accuracy.

### Future Work
- GARCH / EGARCH extensions for volatility modelling
- LSTM / Transformer deep learning comparison
- Sentiment analysis from social media as an exogenous regressor (ARIMAX / VARX)
- Bayesian VAR (BVAR) with informative priors


In [ ]:
# Save forecast results
os.makedirs(os.path.join("..", "results"), exist_ok=True)
comparison.round(2).to_csv(os.path.join("..", "results", "forecast_summary.csv"))
print("Forecast summary saved to results/forecast_summary.csv")